# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [3]:
lamp_review = """
Kellett egy szép lámpa a hálószobámba, és ez volt \
további tárhely és nem túl magas ár. \
Gyorsan megvan. A lámpánk húrja elszakadt a \
tranzit és a cég boldogan küldött át egy újat. \
Pár napon belül meg is jött. Könnyű volt feltenni \
együtt. Hiányzó alkatrészem volt, ezért felvettem a kapcsolatot a \
támogatást és nagyon gyorsan megkaptam a hiányzó darabot! \
Szerintem a Lumina egy nagyszerű társaság, aki törődik vele \
vásárlóikról és termékeikről!!
"""

## Sentiment (positive/negative)

In [4]:
prompt = f"""
Mi a véleménye a következő termékismertetőről,
amelyet hármas backtickkel határolnak?

Vélemény szövege: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Mivel az értékelés csak pozitívumokat említ, így azt gondolom, hogy a termékismertető eléggé elfogult és nem ad objektív véleményt a termékről. Azonban, ha a termék tényleg jó minőségű és a cég valóban ilyen jó szolgáltatást nyújt, akkor az értékelés valószínűleg igaz lehet. Azonban, ha valaki az értékelés alapján dönt a termék megvásárlásáról, akkor érdemes lehet további véleményeket is keresni, hogy teljes képet kapjon a termékről és a cég szolgáltatásairól.


In [5]:
prompt = f"""
Mi a véleménye a következő termékismertetőről,
amelyet hármas backtickkel határolnak?

Válaszát egyetlen szóban adja meg, vagy "pozitív" \
vagy "negatív".

Vélemény szövege: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Pozitív.


## Identify types of emotions

In [6]:
prompt = f"""
Határozza meg azon érzelmek listáját, amelyeket a \
az alábbi áttekintés kifejezi. Legfeljebb \
öt elem a listában. Formázza válaszát \
vesszővel elválasztott kisbetűs szavak.

Szöveg áttekintése: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

boldogság, elégedettség, megkönnyebbülés, elégedettség, bizalom


## Identify anger

In [7]:
prompt = f"""
Haragot fejez ki a következő értékelés írója?\
Az áttekintést háromszoros backtick határolja. \
Válaszát igennel vagy nemmel adja meg.

Szöveg áttekintése: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Igen.


## Extract product and company name from customer reviews

In [8]:
prompt = f"""
Azonosítsa a következő elemeket a vélemény szövegéből:
- A lektor által vásárolt elem
- A tételt gyártó cég

Az áttekintést háromszoros backtick határolja. \
Formázza válaszát JSON-objektumként a következővel: \
A "cikk" és a "márka" a kulcsok.
Ha az információ nem található meg, használja az "ismeretlen" kifejezést \
mint az érték.
Legyen a lehető legrövidebb válasz.
  
Szöveg áttekintése: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "cikk": "lámpa",
  "márka": "Lumina"
}
```


## Doing multiple tasks at once

In [9]:
prompt = f"""
Azonosítsa a következő elemeket a vélemény szövegéből:
- hangulat (pozitív vagy negatív)
- Haragot fejez ki a bíráló? (igaz vagy hamis)
- A lektor által vásárolt elem
- A tételt gyártó cég

Az áttekintést háromszoros backtick határolja. \
Formázza válaszát JSON-objektumként a következővel: \
A "érzelem", "harag", "cikk" és "márka" a kulcsok.
Ha az információ nem található meg, használja az "ismeretlen" kifejezést \
mint az érték.
Legyen a lehető legrövidebb válasz.
Formázza az Anger értéket logikai értékként.

Szöveg áttekintése: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "érzelem": "pozitív",
  "harag": false,
  "cikk": "lámpa",
  "márka": "Lumina"
}
```


## Inferring topics

In [10]:
story = """
A kormány által nemrégiben végzett felmérés szerint
a közszférában dolgozókat felkérték, hogy értékeljék szintjüket
elégedettek az osztállyal, ahol dolgoznak.
Az eredményekből kiderült, hogy a NASA volt a legnépszerűbb
osztály 95%-os elégedettségi osztályzattal.

A NASA egyik alkalmazottja, John Smith kommentálta az eredményeket:
kijelentve: „Nem vagyok meglepve, hogy a NASA végzett az élen.
Ez egy nagyszerű hely, ahol csodálatos emberekkel és
hihetetlen lehetőségeket. Büszke vagyok rá, hogy része lehetek
egy ilyen innovatív szervezet."

Az eredményeket a NASA vezetősége is üdvözölte,
Tom Johnson rendező kijelentette: „Nagyon örülünk
hallottam, hogy alkalmazottaink elégedettek a NASA-nál végzett munkájukkal.
Tehetséges és elhivatott csapatunk van, akik fáradhatatlanul dolgoznak
hogy elérjük céljainkat, és fantasztikus látni, hogy az övék
a kemény munka kifizetődik."

A felmérésből az is kiderült, hogy a
A társadalombiztosítási hivatal volt a legalacsonyabb elégedettséggel
besorolással, az alkalmazottak mindössze 45%-a jelezte, hogy igen
elégedettek a munkájukkal. A kormány vállalta
kezelni a munkavállalók által a felmérés során felvetett aggályokat, és
törekedni kell a munkával való elégedettség javítására minden osztályon.
"""

## Infer 5 topics

In [11]:
prompt = f"""
Határozzon meg öt témát, amelyekről a \
következő szöveg, amelyet háromszoros backtick határol.

Minden elem legyen egy vagy két szó hosszú.

Válaszát vesszővel elválasztott elemek listájaként formázza.

Szövegminta: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Felmérés eredményei, NASA elégedettségi osztályzata, Társadalombiztosítási hivatal elégedettségi osztálya, Alkalmazottak véleménye, Munkahelyi elégedettség javítása


In [12]:
response.split(sep=',')

['Felmérés eredményei',
 ' NASA elégedettségi osztályzata',
 ' Társadalombiztosítási hivatal elégedettségi osztálya',
 ' Alkalmazottak véleménye',
 ' Munkahelyi elégedettség javítása']

In [13]:
topic_list = [
"nasa", "helyi önkormányzat", "mérnökség",
     "munkavállalók elégedettsége", "szövetségi kormány"
]

## Make a news alert for certain topics

In [14]:
prompt = f"""
Határozza meg, hogy a következő listában szereplő \
témák egy téma az alábbi szövegben, amely
hármas backtickkel határolják.

Válaszát listában adja meg 0-val vagy 1-gyel minden témakörhöz.\

Témakörök listája:{", ".join(topic_list)}

minta szöveg: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
helyi önkormányzat: 0
mérnökség: 0
munkavállalók elégedettsége: 1
szövetségi kormány: 1


In [15]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Try experimenting on your own!